## sentiment analysis with LSTM

In [158]:
import pandas as pd
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dropout ,Dense ,LSTM , Embedding
from sklearn.model_selection import train_test_split
import numpy as np

In [139]:
data = pd.read_csv("C:\\Users\\DU357159\\Documents\\ENU_DATA\\data_ml\\first-gop-debate-twitter-sentiment\\Sentiment.csv")

In [140]:
data = data[["text" , "sentiment"]]

In [141]:
data = data[data["sentiment"] != "Neutral"]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [142]:
print(data[ data['sentiment'] == 'Positive'].size)
print(data[ data['sentiment'] == 'Negative'].size)

4472
16986


In [143]:
data['text'] = data["text"].apply(lambda x :x.replace('rt' ,' '))

In [144]:
max_features = 2000
tokenize = Tokenizer(num_words = max_features, split = ' ')
tokenize.fit_on_texts(data.text.values)
X = tokenize.texts_to_sequences(data.text.values)
X = pad_sequences(X)

In [145]:
print(X.shape)

(10729, 28)


In [146]:
model = Sequential()

In [147]:
embed_dim = 128
lstm_out = 196
model.add(Embedding(max_features , embed_dim ,input_length = X.shape[1]))
model.add(LSTM(lstm_out))
model.add(Dropout(0.2))
model.add(Dense(2 , activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

In [148]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 28, 128)           256000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dropout_4 (Dropout)          (None, 196)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________


In [169]:
Y = pd.get_dummies(data["sentiment"])


In [150]:
x_train ,x_test ,y_train,y_test = train_test_split(X ,Y ,test_size = 0.3 ,random_state =4)

In [151]:
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(7510, 28) (7510, 2)
(3219, 28) (3219, 2)


In [152]:
batch_size =32

In [154]:
model.fit(x_train ,y_train ,epochs = 7 ,verbose =1 ,batch_size = 32 )

Epoch 1/7
7510/7510 [==============================] - 24s 3ms/step - loss: 0.2687 - acc: 0.8897: 2s - los - ETA: 0s - loss: 0.2677 - acc: 0.89
Epoch 2/7
7510/7510 [==============================] - 24s 3ms/step - loss: 0.2314 - acc: 0.9067
Epoch 3/7
7510/7510 [==============================] - 25s 3ms/step - loss: 0.1999 - acc: 0.9158
Epoch 4/7
7510/7510 [==============================] - 24s 3ms/step - loss: 0.1732 - acc: 0.9285
Epoch 5/7
7510/7510 [==============================] - 25s 3ms/step - loss: 0.1493 - acc: 0.9394
Epoch 6/7
7510/7510 [==============================] - 24s 3ms/step - loss: 0.1327 - acc: 0.9454
Epoch 7/7
7510/7510 [==============================] - 23s 3ms/step - loss: 0.1161 - acc: 0.9527


In [165]:
score ,acc = model.evaluate(x_test ,y_test)

3219/3219 [==============================] - 2s 738us/step


In [166]:
print("score" ,score)
print("acc" , acc)

score 0.6736308456615248
acc 0.8247903076600273


In [167]:
twt = ['Meetings: Because none of us is as dumb as all of us.']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenize.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0  206  633    6  150    5   55 1055   55   46    6  150]]
negative


#### WORD EMBEDDING
Enter word embeddings – word embeddings try to “compress” large one-hot word vectors into much smaller vectors (a few hundred elements) which preserve some of the meaning and context of the word. Word2Vec is the most common process of word embedding and will be explained below.